In [1]:
import csv
import numpy as np
import pandas as pd
import random
import json
import requests
from sklearn.preprocessing import StandardScaler

In [14]:
## set credentials and endpoint
username = "default\\admin"
password = "Test123!"
url = "https://vsystem.ingress.dh-rzsmhj7iwkz.di-dev2.shoot.canary.k8s-hana.ondemand.com/app/pipeline-modeler/openapi/service/1818e767-f6fe-4215-bdfb-b3679ac62bf4/"
path_predict_proba = "v1/predict_proba/"
path_predict_classes = "v1/predict_classes/"

def predict(test, predict_classes=True):
    global username, password, url, numberClassifications
    # call the REST api to predict
    headers = {
                'Content-Type': 'application/json',
                'X-Requested-With': 'XMLHttpRequest'
              }
    payload = json.loads('{"data": ' + str(test[0:random.randrange(1,10,2)].tolist()) + '}')
    print(payload)
    url2 = ""
    if predict_classes:
        url2 = url + path_predict_classes
    else:
        url2 = url + path_predict_proba
    resp = requests.post(url2, auth=(username, password), headers=headers, json=payload)

    return resp

In [6]:
test_df = pd.read_csv('titanic_test.csv')
X_test = test_df.drop("alone", axis = 1).to_numpy()
str(X_test[0:random.randrange(1,10,2)].tolist())

'[[3, 0, 2, 0, 0, 1, 1, 1]]'

In [7]:
payload = '{"data": ' + str(X_test[0:3].tolist()) + '}'
feed = json.loads(payload)
input_data = np.array(feed['data'])
scaler = StandardScaler()
# scaler.fit(X_test)
# X_test = scaler.transform(input_data)
X_test

array([[3, 0, 2, ..., 1, 1, 1],
       [2, 0, 0, ..., 0, 0, 0],
       [3, 1, 0, ..., 0, 0, 0],
       ...,
       [3, 0, 2, ..., 0, 1, 1],
       [1, 1, 1, ..., 1, 2, 0],
       [3, 0, 0, ..., 2, 1, 1]])

In [15]:
response = predict(X_test, False)
print(response.text)
for prediction in json.loads(response.text)['Results']:
    print(prediction)

{'data': [[3, 0, 2, 0, 0, 1, 1, 1], [2, 0, 0, 1, 1, 0, 0, 0], [3, 1, 0, 3, 2, 0, 0, 0], [3, 1, 2, 0, 1, 0, 2, 0], [3, 1, 4, 0, 1, 0, 2, 0], [1, 1, 6, 1, 3, 1, 2, 0], [3, 0, 6, 0, 1, 0, 1, 1]]}
{"Results": [[0.8886315479050001, 0.11136845209499993], [0.008316723808670656, 0.9916832761913293], [0.795118419314601, 0.20488158068539902], [0.4066056965370233, 0.5933943034629767], [0.6115256865343618, 0.3884743134656382], [0.009357639604845813, 0.9906423603951542], [0.9541847552087508, 0.045815244791249266]]}
[0.8886315479050001, 0.11136845209499993]
[0.008316723808670656, 0.9916832761913293]
[0.795118419314601, 0.20488158068539902]
[0.4066056965370233, 0.5933943034629767]
[0.6115256865343618, 0.3884743134656382]
[0.009357639604845813, 0.9906423603951542]
[0.9541847552087508, 0.045815244791249266]
